In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import re

In [16]:
tsens_df = pd.read_csv('report', sep=r' \| ', engine='python').query('algo=="TSens" & scale == 0.01 & query == "q3"')[['algo', 'all_table_tstar']]
elastic_df = pd.read_csv('report_elastic', sep=r' \| ', engine='python').query('algo=="Elastic" & scale == 0.01 & query == "q3"')[['algo', 'all_table_tstar']]

In [17]:
df = pd.concat([tsens_df, elastic_df])
df

,algo,all_table_tstar
37,TSens,"[(REGION, [('regionkey', 2)], 647), (NATION, [..."
8,Elastic,"[(REGION, [('REGIONKEY', 3)], 120350000, 429),..."


In [45]:
tsens_df = df.query('algo == "TSens"')['all_table_tstar'].tolist()[0]
elstc_df = df.query('algo == "Elastic"')['all_table_tstar'].tolist()[0]
elstc_df

"[(REGION, [('REGIONKEY', 3)], 120350000, 429), (SUPPLIER, [('NATIONKEY', 10), ('SUPPKEY', 38)], 51000000, 25), (NATION, [('NATIONKEY', 24), ('REGIONKEY', 3)], 24070000, 129), (CUSTOMER, [('NATIONKEY', 24), ('CUSTKEY', 898)], 11200000, 9), (PART, [('PARTKEY', 286)], 2550000, 3), (PARTSUPP, [('SUPPKEY', 55), ('PARTKEY', 1798)], 637500, 0), (ORDERS, [('ORDERKEY', 31297), ('CUSTKEY', 898)], 350000, 0), (LINEITEM, [('ORDERKEY', 48993), ('SUPPKEY', 55), ('PARTKEY', 1798)], 50000, 0)]"

In [47]:
from collections import defaultdict
relns_tstar = defaultdict(list)

for all_table_tstar in [tsens_df, elstc_df]:
    data = str(all_table_tstar)
    data = re.sub(r'\((\w+),', r'("\1",', data)
    data = eval(data)
    for table_tstar in data:
        if len(table_tstar) == 3:
            reln, tupl, sens = table_tstar
            tupl = ', '.join('%s(%s)'%(t[0], t[1]) for t in tupl)
            relns_tstar[reln] += [tupl, sens]
        else:
            reln, tupl, esens, tsens = table_tstar
            tupl = ', '.join('%s(%s)'%(t[0].lower(), t[1]) for t in tupl)
            relns_tstar[reln] += [tupl, esens, tsens]
            
for reln, reln_tstar in relns_tstar.items():
    print(reln, *reln_tstar, sep='\t')
        

REGION	regionkey(2)	647	regionkey(3)	120350000	429
NATION	regionkey(4), nationkey(16)	179	nationkey(24), regionkey(3)	24070000	129
SUPPLIER	suppkey(51), nationkey(3)	46	nationkey(10), suppkey(38)	51000000	25
CUSTOMER	nationkey(16), custkey(154)	18	nationkey(24), custkey(898)	11200000	9
PART	partkey(1311)	7	partkey(286)	2550000	3
ORDERS	orderkey(57410), custkey(117)	5	orderkey(31297), custkey(898)	350000	0
PARTSUPP	partkey(1580), suppkey(81)	4	suppkey(55), partkey(1798)	637500	0
LINEITEM		1	orderkey(48993), suppkey(55), partkey(1798)	50000	0
